In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from skopt import BayesSearchCV

import tensorflow as tf

from src.utils.models import define_model, train_and_pred
from src.utils.data import load_data

pd.options.display.max_columns = 100
%matplotlib inline
plt.style.use('seaborn')

/home/hardian_lawi/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/hardian_lawi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
radiant_cols = ['hero_' + str(i) for i in range(5)]
dire_cols = ['hero_' + str(i) for i in range(5, 10)]
no_heroes = 116

radiants, dires, df = load_data()

In [3]:
n_splits = 5
kf = KFold(n_splits, shuffle=True, random_state=10)

print('=================================================')

avg_score = {}
scores = {'model': [], 'model_1': []}
losses = {'model': [], 'model_1': []}

l_values = [0.001, 0.01, 0.5, 1, 1.5, 2, 3, 7, 10, 15, 20, 25]

for l in l_values:
    
    print(l, ' ', end='')
    
    C = 1 / l
    
    score = dict(model=0, model_1=0)
    loss = dict(model=0, model_1=0)
    
    for i, (train_idx, test_idx) in enumerate(kf.split(df)):

        model = LogisticRegression(random_state=0, solver='saga', C=C, penalty='l2', n_jobs=4, max_iter=200)
        model_1 = LogisticRegression(random_state=0, solver='saga', C=C, penalty='l1', n_jobs=4, max_iter=200)
        
        X_train = np.concatenate([radiants.iloc[train_idx].copy(), dires.iloc[train_idx].copy()], axis=1)
        X_test = np.concatenate([radiants.iloc[test_idx].copy(), dires.iloc[test_idx].copy()], axis=1)

        y_train = df.radiant_win.iloc[train_idx].values
        y_test = df.radiant_win.iloc[test_idx].values
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        model.fit(X_train, y_train)
        model_1.fit(X_train, y_train)
        
        ypreds = model.predict_proba(X_test)[:, 1]
        ypreds_1 = model_1.predict_proba(X_test)[:, 1]
        
        score['model'] += accuracy_score(y_test, ypreds > 0.5)
        loss['model'] += log_loss(y_test, ypreds)
        
        score['model_1'] += accuracy_score(y_test, ypreds_1 > 0.5)
        loss['model_1'] += log_loss(y_test, ypreds_1)
        
        
    for k in ['model', 'model_1']:
        
        score[k] /= n_splits
        scores[k].append(score[k])

        loss[k] /= n_splits
        losses[k].append(loss[k])
    
print('\nDone.')
          
############################################################################

# print('Avg threshold', avg_threshold / (n_splits * len(models_dict)), '\n\n')
# print('Number of features:', len(feature_columns))
# print('Features:', feature_columns, '\n\n')

0.001  

/home/hardian_lawi/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.01  0.5  1  1.5  2  3  7  10  15  20  25  
Done.
